In [24]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType


In [25]:
# Prepare data
start = '2019-01-01 UTC'  # crypto is in UTC
end = datetime.utcnow()
dir_name = '/Users/andre/Documents/Python/trading_clone_singapore_DO/trading_singapore_digitalocean/vectorBT'
os.makedirs(dir_name, exist_ok=True)

# Function to download and save data
def download_and_save_data(symbol, file_name, overwrite=False):
    joined = os.path.join(dir_name, file_name)
    if os.path.exists(joined) and not overwrite:
        data = pd.read_pickle(joined)
        print(f'File {file_name} exists already')
    else:
        data = vbt.CCXTData.download(exchange='kucoin', symbols=symbol, timeframe='1d', start=start, end=end).get()
        data.to_pickle(joined)
        print(f'Downloaded and saved {symbol} data')
    return data

# List of assets
assets = ['BTC', 'ETH']
divider = '/'  # Divider for the assets
base_value = '/USDT'  # Base value for the assets

# Dictionary to store data
data_dict = {}

# Download data for each asset
for asset in assets:
    symbol = asset.lower()
    file_name = f'{symbol}_price_1d.pkl'
    data_dict[symbol] = download_and_save_data(asset + base_value, file_name, overwrite=False)

# Align the DataFrames by their index
aligned_data = [data_dict[symbol].align(data_dict[assets[0].lower()], join='inner')[0] for symbol in data_dict]

# Concatenate the aligned DataFrames
comb_price = pd.concat(aligned_data, axis=1, keys=[asset for asset in assets])
comb_price.columns.names = ['symbol', 'attribute']

# Create DataFrames for Open, High, Low, and Close prices
comb_open = pd.concat([data_dict[symbol]['Open'] for symbol in data_dict], axis=1, keys=[f'{symbol}_open' for symbol in data_dict])
comb_high = pd.concat([data_dict[symbol]['High'] for symbol in data_dict], axis=1, keys=[f'{symbol}_high' for symbol in data_dict])
comb_low = pd.concat([data_dict[symbol]['Low'] for symbol in data_dict], axis=1, keys=[f'{symbol}_low' for symbol in data_dict])
comb_close = pd.concat([data_dict[symbol]['Close'] for symbol in data_dict], axis=1, keys=[f'{symbol}_close' for symbol in data_dict])

# Display the resulting DataFrames
print("Open Prices:")
print(comb_open.head())

print("\nHigh Prices:")
print(comb_high.head())

print("\nLow Prices:")
print(comb_low.head())

print("\nClose Prices:")
print(comb_close.head())


File btc_price_1d.pkl exists already
File eth_price_1d.pkl exists already
Open Prices:
                              btc_open    eth_open
Open time                                         
2019-01-01 00:00:00+00:00  3700.170853  131.361407
2019-01-02 00:00:00+00:00  3799.644687  139.098781
2019-01-03 00:00:00+00:00  3857.633097  152.009074
2019-01-04 00:00:00+00:00  3767.854300  146.145033
2019-01-05 00:00:00+00:00  3789.000000  152.536302

High Prices:
                              btc_high    eth_high
Open time                                         
2019-01-01 00:00:00+00:00  3805.841739  139.826115
2019-01-02 00:00:00+00:00  3880.257733  155.000000
2019-01-03 00:00:00+00:00  3862.600830  153.584211
2019-01-04 00:00:00+00:00  3821.195343  154.700000
2019-01-05 00:00:00+00:00  3838.674540  158.871870

Low Prices:
                               btc_low     eth_low
Open time                                         
2019-01-01 00:00:00+00:00  3645.525763  130.082442
2019-01-02 00:00:00

## ma cross overall trend prediction

In [26]:
def ma_cross_rsi(close_price, ma1_window, ma2_window, ma3_window, ma4_window, ma5_window, rsi_buy_threshold, rsi_sell_threshold):
    # Calculate moving averages
    ma1 = vbt.MA.run(close_price, window=ma1_window).ma.to_numpy()
    ma2 = vbt.MA.run(close_price, window=ma2_window).ma.to_numpy()
    ma3 = vbt.MA.run(close_price, window=ma3_window).ma.to_numpy()
    ma4 = vbt.MA.run(close_price, window=ma4_window).ma.to_numpy()
    ma5 = vbt.MA.run(close_price, window=ma5_window).ma.to_numpy()
    
    # Determine slow trend direction
    slow_trend_up = (ma4 > ma5)
    slow_trend_down = (ma4 < ma5)

    # Determine short trend direction
    short_trend_up = (ma1 > ma2) & (ma2 > ma3)
    short_trend_undecided = (ma1 < ma2) & (ma2 > ma3)
    short_trend_down = (ma1 < ma2) & (ma2 < ma3)

    # Calculate RSI and determine buy/sell signals
    rsi = vbt.RSI.run(close_price, window=14).rsi.to_numpy()
    rsi_buy = rsi > rsi_buy_threshold
    rsi_sell = rsi < rsi_sell_threshold
    
    # Generate buy and sell signals
    signal_buy = slow_trend_up & short_trend_up & rsi_buy
    signal_sell = slow_trend_down & short_trend_down & rsi_sell

    # Define caution signals for long and short positions
    no_caution_long = slow_trend_up & short_trend_up
    caution_long1 = slow_trend_up & short_trend_undecided 
    caution_long2 = slow_trend_up & short_trend_down

    no_caution_short = slow_trend_down & short_trend_down
    caution_short1 = slow_trend_down & short_trend_undecided
    caution_short2 = slow_trend_down & short_trend_up

    # Define conditions and corresponding values for signals
    conditions = [
        signal_buy,
        no_caution_long,
        caution_long1,
        caution_long2,
        signal_sell,
        no_caution_short,
        caution_short1,
        caution_short2
    ]

    values = [
        100,  # Strong buy signal (slow trend up, short trend up, and RSI above the buy threshold)
        75,   # No caution long (slow trend up and short trend up)
        50,   # Caution long 1 (slow trend up and short trend undecided)
        25,   # Caution long 2 (slow trend up and short trend down)
        -100, # Strong sell signal (slow trend down, short trend down, and RSI below the sell threshold)
        -75,  # No caution short (slow trend down and short trend down)
        -50,  # Caution short 1 (slow trend down and short trend undecided)
        -25   # Caution short 2 (slow trend down and short trend up)
    ]

    # Use np.select to apply the conditions and assign the corresponding values
    signal = np.select(conditions, values, default=0)  # 0: No signal (default value when none of the conditions are met)

    return signal

# Create the indicator factory
ma_cross_slow = vbt.IndicatorFactory(
    class_name='MA_Cross_rsi',
    short_name='ma_c_slow',
    input_names=['Close'],
    param_names=['ma1', 'ma2', 'ma3', 'ma4', 'ma5', 'rsi_buy_threshold', 'rsi_sell_threshold'],
    output_names=['ma_cross_rsi']
).from_apply_func(ma_cross_rsi, ma1=20, ma2=30, ma3=40, ma4=125, ma5=150, rsi_buy_threshold=70, rsi_sell_threshold=30)

# # Define window sizes and RSI thresholds
# ma_trend_fast = 50
# ma_trend_mid = 100
# ma_trend_slow = 150

# ma_slow_trend1 = 400
# ma_slow_trend_2 = 500

# rsi_buy_threshold = [70,60]
# rsi_sell_threshold = [20,30]


# Define window sizes and RSI thresholds using np.arange
ma_trend_fast = np.arange(20, 51,5)  # Single value range for consistency
ma_trend_mid = np.arange(50, 101,10)  # Single value range for consistency
ma_trend_slow = np.arange(150, 151,10)  # Single value range for consistency

ma_slow_trend1 = np.arange(400, 401,20)  # Single value range for consistency
ma_slow_trend_2 = np.arange(500, 501,20)  # Single value range for consistency

rsi_buy_threshold = np.arange(60, 71, 10)  # Range from 60 to 70 with step 10
rsi_sell_threshold = np.arange(20, 31, 10)  # Range from 20 to 30 with step 10


# Run the indicator
trend = ma_cross_slow.run(
    comb_close,
    ma1=ma_trend_fast,
    ma2=ma_trend_mid,
    ma3=ma_trend_slow,
    ma4=ma_slow_trend1,
    ma5=ma_slow_trend_2,
    rsi_buy_threshold=rsi_buy_threshold,
    rsi_sell_threshold=rsi_sell_threshold,
    param_product=True
)

print(len(assets))
print(len(trend.ma_cross_rsi.columns))
# Display the first 150 rows of the signal output
needed_copies = len(trend.ma_cross_rsi.columns)/ len(assets)

2
336


# amke copies to match possible strategies 

In [27]:

n = int(needed_copies) 
# Create DataFrames for Open, High, Low, and Close prices with n copies
comb_open = pd.concat([data_dict[symbol]['Open'].rename(f'{symbol}_open{i+1}') for symbol in data_dict for i in range(n)], axis=1)
comb_high = pd.concat([data_dict[symbol]['High'].rename(f'{symbol}_high{i+1}') for symbol in data_dict for i in range(n)], axis=1)
comb_low = pd.concat([data_dict[symbol]['Low'].rename(f'{symbol}_low{i+1}') for symbol in data_dict for i in range(n)], axis=1)
comb_close = pd.concat([data_dict[symbol]['Close'].rename(f'{symbol}_close{i+1}') for symbol in data_dict for i in range(n)], axis=1)

# Display the resulting DataFrames with n copies
print("Open Prices with n copies:")
print(comb_open.head())

print("\nHigh Prices with n copies:")
print(comb_high.head())

print("\nLow Prices with n copies:")
print(comb_low.head())

print("\nClose Prices with n copies:")
print(comb_close.head())

Open Prices with n copies:
                             btc_open1    btc_open2    btc_open3    btc_open4  \
Open time                                                                       
2019-01-01 00:00:00+00:00  3700.170853  3700.170853  3700.170853  3700.170853   
2019-01-02 00:00:00+00:00  3799.644687  3799.644687  3799.644687  3799.644687   
2019-01-03 00:00:00+00:00  3857.633097  3857.633097  3857.633097  3857.633097   
2019-01-04 00:00:00+00:00  3767.854300  3767.854300  3767.854300  3767.854300   
2019-01-05 00:00:00+00:00  3789.000000  3789.000000  3789.000000  3789.000000   

                             btc_open5    btc_open6    btc_open7    btc_open8  \
Open time                                                                       
2019-01-01 00:00:00+00:00  3700.170853  3700.170853  3700.170853  3700.170853   
2019-01-02 00:00:00+00:00  3799.644687  3799.644687  3799.644687  3799.644687   
2019-01-03 00:00:00+00:00  3857.633097  3857.633097  3857.633097  3857.633097   


In [ ]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices):
    close_price = c.close[c.i, c.col]
    low_price = low[c.i, c.col]
    high_price = high[c.i, c.col]
    open_price = open_[c.i, c.col]    
    print("INDEX", c.i)
    print("COL", c.col)
    print('close price:', close_price)
    print('high price:', high_price)
    print('low price:', low_price)
    print('open price:', open_price)
    print('position size :', c.position_now)
    print('cash:', c.cash_now)
    print('entries:', entries[c.i, c.col])
    print()

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        if low[c.i-1,c.col] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=sl_prices[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly)

            print('sl hit at Index:', c.i-1)
            print('sl price:', sl_prices[c.i])
            return value
        
        # check condition to adjust sl 
        # find lowest low in the last 5 bars
        # set sl n% lower than the lowest low
        # if new sl is higher than the current sl update
        # if new sl is lower than the current sl do nothing

        # sl update for long position
        if c.i >= 5:  # Ensure there are enough bars to look back
            lowest_low = np.min(low[c.i-2:c.i,c.col])  # Find the lowest low in the last 5 bars
            if (entries[c.i, c.col] == 75):
                print('sl update before',c.i,sl_prices[c.i])
                update1 = lowest_low * 0.90
                if update1 > sl_prices[c.i]:
                    sl_prices[:]= update1
                    print('sl update after at index',c.i,sl_prices[c.i])
            if (entries[c.i, c.col] == 50):  
                update2 = lowest_low * 0.95
                if update2 > sl_prices[c.i]:
                    print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= update2          
                print('sl update after at index',c.i,sl_prices[c.i])
            if (entries[c.i, c.col] == 25):
                update3 = lowest_low * 0.98
                if update3 > sl_prices[c.i]:
                    print('sl update before',c.i,sl_prices[c.i])
                    update3 = lowest_low * 0.98
                print('sl update after at index',c.i,sl_prices[c.i])

    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i, c.col] == 100:
            sl_price = close_price * 0.90
            order = vbt.portfolio.nb.order_nb(
                size=0.1,  # Adjusted order size
                price=close_price,  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.0,  # No fees
                slippage=0.0,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )
            sl_prices[:] = sl_price 
            print('sl to put in array price:', sl_price)
            print('sl array',sl_prices[1])   # Save the SL price using a single index
            print('long order', c.i,order)
            print()
            return order
    


    return vbt.portfolio.enums.NoOrder


open_ = comb_open.to_numpy()
high = comb_high.to_numpy()
low = comb_low.to_numpy()
close = comb_close
entries = trend.ma_cross_rsi.to_numpy()

# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,  # Pass the SL prices array
    init_cash=100000,  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable

# Optional: Plot equity curve
# pf.plot().show()



In [23]:
check = pf.orders.records_readable.sort_values(by='Timestamp')
check
# print total return
pf.total_return()

btc_close1    0.132070
btc_close2    0.266797
btc_close3    0.132070
btc_close4    0.266797
eth_close1    0.204281
eth_close2    0.207684
eth_close3    0.204281
eth_close4    0.207684
Name: total_return, dtype: float64